In [ ]:

%pip install pydub

In [ ]:
#yolov4-tiny without mail
import cv2
import pywhatkit as pw
import datetime
import time
import os
from pydub import AudioSegment
import numpy as np  
import pyautogui as pg


import base64
import simpleaudio as sa


# wave_obj = sa.WaveObject.from_wave_file("siren-alert-96052.wav")
# set up YOLOv4 net
net = cv2.dnn.readNetFromDarknet("/home/king/Downloads/yolov4-tiny.cfg", "/home/king/Downloads/yolov4-tiny.weights")
classes = []
with open("/home/king/Downloads/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in [net.getUnconnectedOutLayers()]]


# initialize the video capture device
cap = cv2.VideoCapture(0)
# set the width of the frames to 640
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)

# set the height of the frames to 480
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# set the frame rate to 30 fps
cap.set(cv2.CAP_PROP_FPS, 30)


mp3_file = "/home/king/Downloads/siren-alert-96052.mp3"
wav_file = "/home/king/Downloads/siren-alert-96052.wav"
audio = AudioSegment.from_mp3(mp3_file)

# convert the audio to WAV
audio.export(wav_file, format="wav")

alarm_threshold = 4 # number of frames with a person detected required to trigger the alarm
alarm_counter = 0
alarm_triggered = False

message_sent=False
# loop over the frames from the video stream
while True:
    # read a frame from the video stream
    ret, frame = cap.read()
    
    if frame is None:
        break

    # detect objects using YOLOv4-tiny
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    # initialize lists for detected objects
    class_ids = []
    confidences = []
    boxes = []
    
    # process output from YOLOv4-tiny
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w / 2
                y = center_y - h / 2
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # apply non-max suppression to remove redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    indices = np.array(indices)

    # draw bounding boxes and labels for the detected objects
    for i in indices.flatten():
        
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = classes[class_ids[i]]
        if label == 'person':
            #play_obj = wave_obj.play()
            
            cv2.rectangle(frame, tuple(map(int, (x, y))), tuple(map(int, (x + w, y + h))), (0, 0, 255), 2)
            cv2.putText(frame, label, (int(x), int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            cv2.imwrite('human_breach.png', frame)

            # if a person is detected, and the message hasn't been sent yet, send the message and set the flag variable to True
            if not message_sent:
                now=datetime.datetime.now()

                formatted_time = "{}:{}".format(now.hour,now.minute + 1)

                message= "Human movement detected at {}\n".format((now.strftime("%Y-%m-%d %H:%M:%S")))

                
                image = "human_breach.png"
                pw.sendwhats_image("+919458900689", image,message)

               
                message_sent = True


    # add a timestamp to the frame
    cv2.putText(frame, time.strftime("%A %d %B %Y %I:%M:%S%p"), (10, frame.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)





    # add a timestamp to the frame
    cv2.putText(frame, time.strftime("%A %d %B %Y %I:%M:%S%p"), (10, frame.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

    # show the frame
    cv2.imshow("Smart Surveillance System", frame)
    key = cv2.waitKey(1) & 0xFF

      # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break


# release the video capture device and close all windows
cap.release()
cv2.destroyAllWindows()


In [1]:
#FULL PROJECT THE END 
import cv2
import pywhatkit as pw
import datetime
import time
import os
import numpy as np  
import camera
from pydub import AudioSegment
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders
import base64

from_email = 'vsantdasani786@gmail.com'
from_email_password ='lcsyqwnnpbhfrxdc'
to_email = '9theraphotographer@gmail.com'

# set up YOLOv4 net
net = cv2.dnn.readNetFromDarknet("/home/vansh/Downloads/yolov4-tiny.cfg", "/home/vansh/Downloads/yolov4-tiny.weights")
classes = []
with open("/home/vansh/Downloads/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in [net.getUnconnectedOutLayers()]]


# initialize the video capture device
cap = cv2.VideoCapture(0)


# set the width of the frames to 640
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)

# set the height of the frames to 480
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# set the frame rate to 30 fps
cap.set(cv2.CAP_PROP_FPS, 30)


mp3_file = "/home/vansh/Downloads/siren-alert-96052.mp3"
wav_file = "/home/vansh/Downloads/siren-alert-96052.wav"
audio = AudioSegment.from_mp3(mp3_file)

# convert the audio to WAV
audio.export(wav_file, format="wav")

alarm_threshold = 4 # number of frames with a person detected required to trigger the alarm
alarm_counter = 0
alarm_triggered = False

message_sent=False
# loop over the frames from the video stream
while True:
    # read a frame from the video stream
    ret, frame = cap.read()
    
    if frame is None:
        break

    # detect objects using YOLOv4-tiny
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    # initialize lists for detected objects
    class_ids = []
    confidences = []
    boxes = []
    
    # process output from YOLOv4-tiny
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w / 2
                y = center_y - h / 2
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # apply non-max suppression to remove redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    indices = np.array(indices)

    # draw bounding boxes and labels for the detected objects
    for i in indices.flatten():
        
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = classes[class_ids[i]]
        if label == 'person':
            cv2.rectangle(frame, tuple(map(int, (x, y))), tuple(map(int, (x + w, y + h))), (0, 0, 255), 2)
            cv2.putText(frame, label, (int(x), int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            cv2.imwrite('human_breach.png', frame)
             # Compose the email message
            msg = MIMEMultipart()
            msg['From'] = from_email
            msg['To'] = to_email
            msg['Subject'] = "Person Detected!"

            body = "A person has been detected. See the attached photo."
            msg.attach(MIMEText(body, 'plain'))

             # Attach the image to the email
            with open('human_breach.png', "rb") as f:
                part = MIMEBase('application', "octet-stream")
                part.set_payload(f.read())
                encoders.encode_base64(part)
                part.add_header('Content-Disposition', 'attachment', filename='human_breach.png')
                msg.attach(part)

            # Send the email
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(from_email, from_email_password)
            server.sendmail(from_email, to_email, msg.as_string())
            server.quit()
            # if a person is detected, and the message hasn't been sent yet, send the message and set the flag variable to True
            if not message_sent:
                now=datetime.datetime.now()

                formatted_time = "{}:{}".format(now.hour,now.minute + 1)

                message= "Human movement detected at {}".format((now.strftime("%Y-%m-%d %H:%M:%S")))
                
                
                image = "human_breach.png"
                pw.sendwhats_image("+919458900689", image, message)

               
                message_sent = True


# add a timestamp to the frame
    cv2.putText(frame, time.strftime("%A %d %B %Y %I:%M:%S%p"), (10, frame.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

    # show the frame
    cv2.imshow("Smart Surveillance System", frame)
    key = cv2.waitKey(1) & 0xFF

      # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
     os.system.exit()


# release the video capture device and close all windows
cap.release()
cv2.destroyAllWindows()


/home/vansh/miniconda3/envs/tf/lib/python3.9/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'